In [1]:
import os 
import warnings 
import json
import flask_cors, flask
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from diffusers import AutoPipelineForText2Image
import torch
from io import BytesIO
import re
import base64
from PIL import Image 
from dotenv import load_dotenv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import validators

# Load environment variables from .env file
load_dotenv()

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('stopwords')
# Initialize Word_Net_Lemmatizer
lemmatizer = WordNetLemmatizer()
warnings.filterwarnings('ignore')

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
torch.cuda.empty_cache()
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  6.83it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "stabilityai/sd-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
def load_data_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [4]:
num_barrels = 1000
# Load lexicon
lexicon = load_data_from_json(r"Files\\lexicon.json")
# Load the documents
documents = load_data_from_json(r"Files\\documents.json")
barrel_files = os.listdir(r"Files\Barrels")

In [5]:
# Function that takes in content, preprocesses it, 
# and converts it to a list of words
def pre_process_string(content):
    # Remove \n and \t
    content = content.replace('\n', ' ')
    content = content.replace('\t', ' ')
    # Remove all non-characters
    content = re.sub(r'[^a-zA-Z\s]', ' ', content)
    # Remove multiple spaces
    content = re.sub(r'\s+', ' ', content)
    # Convert all characters to lowercase
    content = content.lower()
    # Convert the title into a list of words
    content = content.split()
    # Remove one and two character words
    content = [word for word in content if len(word) > 2]
    # Remove stop_words using nltk
    content = [word for word in content if not word in stopwords.words('english')]
    return content
    
# Function that takes in a list of words and adds them to the lexicon
def build_lexicon(words):
    # Build the lexicon
    new_words = []
    # Look through the words
    for word in words:
        # Lemmatize the word
        word = lemmatizer.lemmatize(word)
        # if that word is not already in lexicon
        if word not in lexicon and word not in new_words:
            # Then add it
            new_words.append(word)
    lexicon.extend(new_words)
    return

# Function to build forward index from raw articles
def build_forward_index(articles):

    # initialize forward_index
    forward_index = dict()

    #initialize documents
    docs = dict()

    # Load the already existing forward_index
    try:
        data = load_data_from_json(r"Files\\forward_index.json")
    except:
        with open(r"Files\\forward_index.json", "w") as file:
            json.dump(dict(), file)
        data = load_data_from_json(r"Files\\forward_index.json")
        
    num_articles = len(documents)
    
    # Extract all urls currently indexed
    try:
        article_urls = [article['url'] for article in documents.values()]
    except:
        article_urls = []
        
    # For each article
    for article in articles:
        # if article is not already forward indexed
        if article['url'] not in article_urls:
            # Pre-process the title and content
            title_words = pre_process_string(article['title'])
            content_words = pre_process_string(article['content'])
            # Update the lexicon
            build_lexicon(title_words + content_words)
            # Lemmatize the words in content and title
            content_words = [lemmatizer.lemmatize(word) for word in content_words]
            title_words = [lemmatizer.lemmatize(word) for word in title_words]
            # Convert the words in title and content to their respective indexes
            content_ids = [lexicon.index(word) for word in content_words]
            title_ids = [lexicon.index(word) for word in title_words]
            # Count the frequencies of words
            frequency = Counter((title_ids * 10) + content_ids)
            forward_index[num_articles] = frequency
            docs[str(num_articles)] = {'title': article['title'], 'url': article['url']}
            # Add the url to the article
            article_urls.append(article['url'])
            num_articles += 1
            
    data.update(forward_index)
    print(docs)
    documents.update(docs)
    # Update the lexicon json file
    with open(r"Files\lexicon.json", "w") as file:
        json.dump(lexicon, file)
    # Update the forward_index json file
    with open(r"Files\forward_index.json", "w") as file:
        json.dump(data, file)
    # Update the documents json file
    with open(r"Files\documents.json", "w") as file:
        json.dump(documents, file)

def build_inverted_index_with_barrels():

    # Load the forward index
    try:
        forward_index = load_data_from_json(r"Files\forward_index.json")
    except:
        return
    
    barrels = [None] * num_barrels

    # Iterate through all articles in the forward_index
    for doc_id, data in forward_index.items():
        # Look at all words in an article
        for word_id in data:
            # Calculate the barrel number for that word
            barrel_no = int(word_id) % num_barrels
            barrel_filename = f"barrel_{str(barrel_no).zfill(5)}.json"
            
            # Check if that barrel exists, if not then create it
            barrel_path = os.path.join(r"Files\Barrels", barrel_filename)
            if not os.path.exists(barrel_path):
                with open(barrel_path, "w") as file:
                    json.dump(dict(), file)
                barrel_files.append(barrel_filename)
            # Load the barrel
            if barrels[barrel_no] is None:
                barrels[barrel_no] = (load_data_from_json(barrel_path))
                
            # If that word is not already in that barrel
            if word_id not in barrels[barrel_no]:
                # Then create a dict at that word_id
                barrels[barrel_no][word_id] = dict()
            # And add the doc_id for that word along with frequency if it is not already there
            if doc_id not in barrels[barrel_no][word_id]:
                barrels[barrel_no][word_id].update({doc_id: data[word_id]})

    # Update the loaded barrels
    for i, barrel in enumerate(barrels):
        if barrel is not None:
            with open(os.path.join(r"Files\Barrels", barrel_files[i]), "w") as file:
                json.dump(barrel, file)
    
    # # Clear the forward_index
    with open(r"Files\forward_index.json", "w") as file:
        json.dump(dict(), file)


def rank_results(search_result): 
     # Rank these documents
    # Sort the dictionary by values (descending order)
    sorted_tuples = sorted(search_result.items(), key=lambda x: x[1], reverse=True)
    
    # Convert the sorted list of tuples back to a dictionary
    ranked_result = dict(sorted_tuples)
    # Extract the article ids
    ranked_articles = ranked_result.keys()
    ranked_articles = list(ranked_articles)
    
    return ranked_articles

In [8]:
app = Flask(__name__)
CORS(app)

@app.route("/search_1", methods=["GET"], endpoint='single_word_search')
def single_word_search():
    word = request.args.get('word')

    # Lemmatize the word
    word = word.lower()
    word = lemmatizer.lemmatize(word)
        
    # Find the id of the word in lexicon
    try:
        word_id = lexicon.index(word)
        # Calculate the barrel of the word
        barrel_no = word_id % num_barrels
        # Load the corresponding barrel
        barrel_filename = f"barrel_{str(barrel_no).zfill(5)}.json"
        barrel_path = os.path.join(r"Files\Barrels", barrel_filename)
        barrel = load_data_from_json(barrel_path)
        # Find out in which documents does the word appear
        search_result = barrel[str(word_id)]
    except:
        search_result = None
    
    if search_result is None: 
        return []

    article_ids = rank_results(search_result)
    titles = [documents[article]['title'] for article in article_ids]
    urls = [documents[article]['url'] for article in article_ids]
    
    json_response = jsonify(article_ids=article_ids, titles=titles, urls=urls)

    return json_response

@app.route("/search_2", methods=["GET"], endpoint='multi_word_search')
def multi_word_search(): 
    query = request.args.get('word')
        # Preprocess the query
    words = pre_process_string(query)

    # Remove those words that are not in lexicon
    words = [word for word in words if word in lexicon]
    # Convert each word to its word_id
    word_ids = [lexicon.index(word) for word in words]
    # Calculate barrel_no of each word
    barrel_nos = [word_id % num_barrels for word_id in word_ids]

    # Load the necessary barrels
    barrels = [None] * num_barrels
    for barrel_no in barrel_nos:
        # If a barrel isn't already loaded, then load it
        if barrels[barrel_no] == None:
            barrel_filename = f"barrel_{str(barrel_no).zfill(5)}.json"
            barrel_path = os.path.join(r"Files\Barrels", barrel_filename)
            barrels[barrel_no] = load_data_from_json(barrel_path)
            
    # Check the first word
    if word_ids:
        result = barrels[barrel_nos[0]][str(word_ids[0])]
        # Check the rest of the words
        for i, word_id in enumerate(word_ids[1:], start = 1):
            # Produce the result for current word
            current_result = barrels[barrel_nos[i]][str(word_id)]
            # Include those articles that are also in the result of current word
            result.update({d:result[d]+current_result[d] for d in result.keys() if d in current_result.keys()})

    if result is None:
        return []
    
    # rank the results
    result = rank_results(result)

    article_ids = result
    titles = [documents[article]['title'] for article in article_ids]
    urls = [documents[article]['url'] for article in article_ids]

    json_response = jsonify(article_ids=article_ids, titles=titles, urls=urls)

    return json_response


@app.route("/gen", methods=["GET"], endpoint='genai_tool')
def genai_tool():
    word = request.args.get('word') 
    image = pipe(prompt=word, num_inference_steps=1, guidance_scale=0.0).images[0]

    image2 = image.convert("RGB")
 
    image_bytes_io = BytesIO()
    image2.save(image_bytes_io, format="PNG")
    image_bytes = image_bytes_io.getvalue()
    
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')  

    json_response = {
        'word': word,
        'image': image_base64
    }

    return jsonify(json_response)

UPLOAD_FOLDER = 'Files\\Uploads'  
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route("/add", methods=["POST"], endpoint='add_content')
def add_content():
    title = request.form.get('title')
    url = request.form.get('url')
    content = request.form.get('content')
    file = request.files.get('file') 
 
    # Check if a file is uploaded
    if file:
        filename = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filename)
        try:
            # Load the file
            data = load_data_from_json(filename) 
        except:
            return jsonify({"message": "Error loading file"}), 500
    else:
        # Check if the url, title and content are correct
        if url and title and content:
            # Validate the url
            if not validators.url(url):
                return jsonify({"message": "Please provide a valid url"}), 400
            # Load the data
            data = [{"title":title, "content":content, "url":url}]
        else:
            return jsonify({"message": "Please provide a file or url, title and content"}), 400

    # build forward and inverted index on it
    try:
        build_forward_index(data)
    except:
        return jsonify({"message": "Error building forward index"}), 500
    
    build_inverted_index_with_barrels()
    
    return jsonify({"message": "Successfully added content"}), 200

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
